In [20]:
# Required Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [21]:
# Load datasets
sector_inter = pd.read_csv('22121000_inter.csv', parse_dates=['RowKey']).set_index('RowKey')
future_data = pd.read_csv('future_data2.csv', parse_dates=['RowKey']).set_index('RowKey')

In [22]:
#check null values
sector_inter.isnull().sum()

INF_Value                 12
precip_lag1                0
precip_lag2                0
precip_lag3                0
precip_lag4                0
precip_lag5                0
precip_lag6                0
precip_lag7                0
precip_rolling_sum         0
precip_rolling_mean        0
precip_rolling_max         0
precip_rolling_min         0
no_rain                    0
light_rain                 0
heavy_rain                 0
days_since_last_rain       0
accumulated_precip_30d     0
precip_change              0
temp_lag1                  0
temp_lag2                  0
temp_lag3                  0
humidity_lag1              0
humidity_lag2              0
humidity_lag3              0
temp_rolling_mean          0
temp_rolling_std           0
humidity_rolling_mean      0
humidity_rolling_std       0
dtype: int64

In [23]:
#fill null values with interpolation linear
sector_inter = sector_inter.interpolate(method='linear', axis=0).ffill().bfill()

In [24]:
# Splitting the data
train, valid = train_test_split(sector_inter, test_size=0.2, shuffle=False)
test = future_data

In [25]:
# Drop INF_Value for XGBoost training and validation
X_train = train.drop('INF_Value', axis=1)
y_train = train['INF_Value']
X_valid = valid.drop('INF_Value', axis=1)
y_valid = valid['INF_Value']

In [26]:
X_test = test.drop('INF_Value', axis=1)

In [27]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

# Define the range of orders and seasonal orders to search through
p_values = [0, 1, 2]  # AutoRegressive order (p)
d_values = [0, 1]     # Differencing order (d)
q_values = [0, 1, 2]  # Moving Average order (q)
seasonal_periods = [7, 30, 13, 12]  # Different seasonal periods to try

best_aic = float("inf")
best_order = (0, 0, 0)
best_seasonal_order = (0, 0, 0, 0)

# Iterate through all possible combinations
for p in p_values:
    for d in d_values:
        for q in q_values:
            for seasonal_period in seasonal_periods:
                order = (p, d, q)
                seasonal_order = (p, d, q, seasonal_period)
                
                # Fit the SARIMA model with the current order and seasonal order
                sarima_model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order)
                sarima_result = sarima_model.fit(disp=False)
                
                # Calculate AIC (Akaike Information Criterion)
                current_aic = sarima_result.aic
                
                # Update the best order if the current AIC is lower
                if current_aic < best_aic:
                    best_aic = current_aic
                    best_order = order
                    best_seasonal_order = seasonal_order

# Print the best SARIMA order and seasonal order
print("Best SARIMA Order:", best_order)
print("Best Seasonal Order:", best_seasonal_order)

# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

Best SARIMA Order: (0, 1, 1)
Best Seasonal Order: (0, 1, 1, 30)


C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\kalman_filter.py:2473: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'


In [28]:
# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\kalman_filter.py:2473: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'


In [29]:
sarima_predictions = abs(sarima_predictions)

In [30]:
future_sarima_predictions = abs(future_sarima_predictions)

In [31]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [50, 100, 200]
}

# Create the XGBoost model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train a new XGBoost model with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params)
best_xgb_model.fit(X_train, y_train)

# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)

Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 50, 'subsample': 0.7}


In [32]:
# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)

In [33]:
# Convert xgb_predictions to a pandas Series with a datetime index
xgb_predictions_series = pd.Series(xgb_predictions, index=y_valid.index)
future_xgb_predictions_series = pd.Series(future_xgb_predictions, index=test.index)


In [58]:
future_ensemble_predictions = 0.1 * future_sarima_predictions + 0.9 * future_xgb_predictions_series


In [59]:
# Ensemble Predictions
ensemble_predictions = 0.1 * sarima_predictions + 0.9 * xgb_predictions_series

In [60]:
# Create DataFrame for Test and Future Predictions
predictions_df = pd.DataFrame({
    'Actual': y_valid,
    'SARIMA_Predictions': sarima_predictions,
    'XGBoost_Predictions': xgb_predictions_series,
    'Ensemble_Predictions': ensemble_predictions
}, index=y_valid.index)

future_predictions_df = pd.DataFrame({
    'SARIMA_Future_Predictions': future_sarima_predictions,
    'XGBoost_Future_Predictions': future_xgb_predictions_series,
    'Ensemble_Future_Predictions': future_ensemble_predictions
}, index=test.index)

In [61]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Actual values
actual_values = predictions_df['Actual']

# SARIMA predictions
sarima_predictions = predictions_df['SARIMA_Predictions']

# XGBoost predictions
xgboost_predictions = predictions_df['XGBoost_Predictions']

# Ensemble predictions
ensemble_predictions = predictions_df['Ensemble_Predictions']

# Calculate MAPE for each method
def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

mape_sarima = mape(actual_values, sarima_predictions)
mape_xgboost = mape(actual_values, xgboost_predictions)
mape_ensemble = mape(actual_values, ensemble_predictions)

# Calculate RMSE for each method
rmse_sarima = np.sqrt(mean_squared_error(actual_values, sarima_predictions))
rmse_xgboost = np.sqrt(mean_squared_error(actual_values, xgboost_predictions))
rmse_ensemble = np.sqrt(mean_squared_error(actual_values, ensemble_predictions))

# Calculate MAE for each method
mae_sarima = mean_absolute_error(actual_values, sarima_predictions)
mae_xgboost = mean_absolute_error(actual_values, xgboost_predictions)
mae_ensemble = mean_absolute_error(actual_values, ensemble_predictions)

# Display the results
print("SARIMA MAPE:", mape_sarima)
print("XGBoost MAPE:", mape_xgboost)
print("Ensemble MAPE:", mape_ensemble)

print("SARIMA RMSE:", rmse_sarima)
print("XGBoost RMSE:", rmse_xgboost)
print("Ensemble RMSE:", rmse_ensemble)

print("SARIMA MAE:", mae_sarima)
print("XGBoost MAE:", mae_xgboost)
print("Ensemble MAE:", mae_ensemble)

SARIMA MAPE: 99.72507543439717
XGBoost MAPE: 90.56600085111184
Ensemble MAPE: 91.12184265488936
SARIMA RMSE: 51.043092275274674
XGBoost RMSE: 46.37988598264237
Ensemble RMSE: 46.32346402054479
SARIMA MAE: 46.06915073001178
XGBoost MAE: 37.28150664834999
Ensemble MAE: 37.795425957079786


In [62]:
predictions_df

,Actual,SARIMA_Predictions,XGBoost_Predictions,Ensemble_Predictions
RowKey,,,,
2022-09-05,160.279996,139.954579,153.559845,152.199315
2022-09-06,159.989996,138.229189,165.212997,162.514615
2022-09-07,165.389996,146.811641,164.277618,162.531018
2022-09-08,171.119996,133.850566,159.859238,157.258363
2022-09-09,175.879996,141.764733,164.944305,162.626348
...,...,...,...,...
2022-12-08,96.829998,126.321646,138.674210,137.438950
2022-12-09,96.399998,125.881826,104.781204,106.891260
2022-12-10,95.569998,128.911758,120.365135,121.219797


In [63]:
future_predictions_df

,SARIMA_Future_Predictions,XGBoost_Future_Predictions,Ensemble_Future_Predictions
RowKey,,,
2022-12-13,138.546299,135.241638,135.572098
2022-12-14,136.296962,130.038422,130.664273
2022-12-15,139.990432,166.826279,164.142689
2022-12-16,141.347981,166.420151,163.912935
2022-12-17,140.132402,122.222702,124.013667
2022-12-18,143.021154,111.091591,114.284545
2022-12-19,142.371040,91.797455,96.854810


In [56]:
#mean of actual values in predictions_df
mean = predictions_df['Actual'].mean()

In [57]:
mean

111.48019952609093